In [ ]:
import wandb
run = wandb.init()
artifact = run.use_artifact('ain-space-org/wandb-registry-model/gaia:v1', type='model')
artifact_dir = artifact.download()
artifact_dir

In [ ]:
import torch
from spherinator.models import (
    ConvolutionalDecoder1D,
    ConvolutionalEncoder1D,
    VariationalAutoencoderPure,
)

encoder = ConvolutionalEncoder1D(input_dim=344, output_dim=3)
decoder = ConvolutionalDecoder1D(input_dim=3, output_dim=344)
model = VariationalAutoencoderPure(encoder=encoder, decoder=decoder)

ckpt = artifact_dir + "/model.ckpt"
checkpoint = torch.load(ckpt, weights_only=True, map_location="cpu")
checkpoint

In [ ]:

model.load_state_dict(checkpoint["state_dict"])
model.eval()

export_options = torch.onnx.ExportOptions(dynamic_shapes=True)
onnx = torch.onnx.dynamo_export(model.variational_encoder, torch.randn(10, 1, 344, device="cpu"), export_options=export_options)
onnx.save("vae_encoder.onnx")

onnx = torch.onnx.dynamo_export(model.decoder, torch.randn(10, 3, device="cpu"), export_options=export_options)
onnx.save("vae_decoder.onnx")

In [ ]:

model.eval()

export_options = torch.onnx.ExportOptions(dynamic_shapes=True)
onnx = torch.onnx.dynamo_export(model.variational_encoder, torch.randn(10, 1, 344, device="cpu"), export_options=export_options)
onnx.save("vae_encoder.onnx")

onnx = torch.onnx.dynamo_export(model.decoder, torch.randn(10, 3, device="cpu"), export_options=export_options)
onnx.save("vae_decoder.onnx")